In [ ]:
import pandas as pd
import numpy as np
import os
import copy
import pickle
import time 
import json
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.algorithms.moo.nsga3 import NSGA3
from pymoo.algorithms.moo.rnsga2 import RNSGA2
from pymoo.algorithms.moo.rnsga3 import RNSGA3
from pymoo.algorithms.soo.nonconvex.ga import GA

from sklearn.ensemble import RandomForestClassifier
from pymoo.algorithms.moo.nsga3 import NSGA3
from pymoo.algorithms.moo.unsga3 import UNSGA3


from pymoo.factory import get_reference_directions
from pymoo.optimize import minimize
from pymoo.visualization.scatter import Scatter
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PolynomialMutation
from pymoo.operators.sampling.rnd import FloatRandomSampling
from sklearn.ensemble import RandomForestRegressor
from pymoo.core.problem import Problem



from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score
from imblearn.metrics import geometric_mean_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import auc
from sklearn.exceptions import NotFittedError

from multiprocessing.pool import ThreadPool

In [ ]:
#global variables 
DATA_PATH = './data/CV_data/eclipse'
FEATURES = [
  'pre', 'ACD', 'FOUT_avg', 'FOUT_max', 'FOUT_sum', 'MLOC_avg',
       'MLOC_max', 'MLOC_sum', 'NBD_avg', 'NBD_max', 'NBD_sum', 'NOF_avg',
       'NOF_max', 'NOF_sum', 'NOI', 'NOM_avg', 'NOM_max', 'NOM_sum', 'NOT',
       'NSF_avg', 'NSF_max', 'NSF_sum', 'NSM_avg', 'NSM_max', 'NSM_sum',
       'PAR_avg', 'PAR_max', 'PAR_sum', 'TLOC', 'VG_avg', 'VG_max', 'VG_sum']
TARGET = 'post'
DATASET_NAME ='eclipse'

In [ ]:
#helpers 
def compute_predictions_probabilities(X,weights) : 
    ready_X = np.ones((X.shape[0],X.shape[1] + 1 ))
    ready_X[:,1:] = X.copy()
    weighted_sum = np.dot(ready_X,weights.T)
    exp_weighted_sum = np.exp(-1*weighted_sum)
    probabilities = 1.0/(exp_weighted_sum + 1.0)
    return probabilities

def predict(probabilities,threshold): 
    predictions = probabilities > threshold 
    return predictions.astype(int)

def recall(y_true,predictions): 
        return recall_score(y_true,predictions)
    
def cost_predictions(costs,predictions,normalized  = False):
    all_costs =  np.sum(predictions*costs)
    if normalized : 
        all_costs = all_costs/np.sum(costs)
    return all_costs
    
def cost_probabilities(costs,probabilities): 
    return np.sum(probabilities*costs)

def benefit(y_true,predictions): 
    return np.sum(predictions*y_true)

def AUC(y_true,probabilities): 
    return roc_auc_score(y_true, probabilities)

def MCC(y_true,predictions): 
    return matthews_corrcoef(y_true, predictions)

def F1(y_true,predictions): 
    return f1_score(y_true, predictions)

def Gmean(y_true,prediction): 
    return geometric_mean_score(y_true, prediction)

def misclassification_cost(y_true,predictions,alpha = 10): 
    tn, fp, fn, tp = confusion_matrix(y_true, predictions).ravel()
    return (fp + alpha*fn)/len(predictions)

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def normalize(x) : 
    return x/np.sum(x)
    
    

In [ ]:
class LearnLrWeights(Problem): 
    def __init__(self,train_data_features,y_true,costs,
                 objectives = ['recall','misclassification_cost'],
                 lb = -10,ub=10,prediction_threshold = 0.5,**kwargs): 
        
        self.train_data_features_df = train_data_features
        nb_variables = len(train_data_features.columns) + 1
        temporary = self.train_data_features_df.to_numpy() 
        self.y_true = y_true
        self.costs = costs
        self.train_data_features_np = np.ones((len(self.train_data_features_df),nb_variables ))
        self.train_data_features_np[:,1:] = temporary
        xl = np.array([lb]*nb_variables)
        xu = np.array([ub]*nb_variables)
        self.prediction_threshold = prediction_threshold
        self.objectives = objectives
        super().__init__(n_var=nb_variables, n_obj=len(objectives), n_constr=0, xl=xl, xu=xu)
        
    def _evaluate(self, x, out, *args, **kwargs):
        pool  = ThreadPool(8)
        F = pool.map(self.evaluate_one_sol, x)
        out['F'] = np.array(F)
    def evaluate_one_sol(self,x):
        out = []
        weighted_sum = np.dot(self.train_data_features_np,x)
        weighted_sum = weighted_sum.astype('float64')
        exp_weighted_sum = np.exp(-1*weighted_sum)
        probabilities = 1.0/(1.0 + exp_weighted_sum)
        predictions = probabilities > self.prediction_threshold 
        predictions = predictions.astype(int)
        for objective in self.objectives: 
            if objective == "recall": 
                out.append(1-self.recall(predictions))
            
            if objective == "TNR": 
                out.append(1-self.TNR(predictions))

            if objective == "churn_cost_predictions":
                out.append(self.churn_cost_predictions(predictions))
            

            if objective == "normalized_churn_cost_predictions":
                out.append(self.churn_cost_predictions(predictions,normalized=True))
                
            if objective == "churn_cost_probabilities":
                out.append(self.churn_cost_probabilities(probabilities))
                
            if objective == "benefit":
                out.append(sum(self.y_true)-self.benefit(predictions)) 
            
            if objective == "AUC":
                #print('TIC')
                out.append(1 - self.AUC(probabilities)) 
            
            if objective == "MCC":
                out.append(-1*self.MCC(predictions)) 
            
            if objective == "F1":
                out.append(-1*self.F1(predictions)) 
            
            if objective == "Gmean":
                out.append(-1*self.Gmean(predictions)) 
            
            if objective == "misclassification_cost": 
                out.append(self.misclassification_cost(predictions)) 

            if objective == 'average_churn_recall' : 
                #print('TOC')
                alpha = 0.5 
                val = alpha*(1-self.recall(predictions)) + (1 - alpha)*self.churn_cost_predictions(predictions,normalized=True)
                out.append(val)

            if objective == "average_AUC_and_recall_churn" : 
                alpha = 0.5
                recall_churn_val = 0.5*(1-self.recall(predictions)) + 0.5*self.churn_cost_predictions(predictions,normalized=True)
                val = alpha*(1 - self.AUC(probabilities)) + (1-alpha)*recall_churn_val
                out.append(val)
            
            if objective == "average_AUC_recall_churn" : 
                val = ((1 - self.AUC(probabilities)) + (1-self.recall(predictions)) + self.churn_cost_predictions(predictions,normalized=True))/3
                out.append(val)
        return out 


    def recall(self,predictions): 
        return recall_score(self.y_true,predictions)
    
    def churn_cost_predictions(self,predictions,normalized = False): 
        return cost_predictions(self.costs,predictions,normalized)
    
    def churn_cost_probabilities(self,probabilities): 
        return np.sum(probabilities*self.costs)
    
    def benefit(self,predictions): 
        return np.sum(predictions*self.y_true)
    
    def AUC(self,probabilities): 
        return roc_auc_score(self.y_true, probabilities)
    
    def MCC(self,predictions): 
        return matthews_corrcoef(self.y_true, predictions)
    
    def F1(self,predictions): 
        return f1_score(self.y_true, predictions)
    
    def Gmean(self,prediction): 
        return geometric_mean_score(self.y_true, prediction)
    
    def misclassification_cost(self,predictions,alpha = 20): 
        tn, fp, fn, tp = confusion_matrix(self.y_true, predictions).ravel()
        return (fp + alpha*fn)/len(predictions)  
    def TNR(self, predictions): 
        return recall_score(self.y_true,predictions,pos_label=0)

In [ ]:
def define_algorithm(algorithm_name,pop_size=400,n_objeectives = 2,
                    crossover_op = SBX( prob=0.5, eta=15),
                    mutation_op = PolynomialMutation(eta=20,prob = 0.1),
                    ref_points = np.array([[0,0]])
                    ): 
    
    if algorithm_name == 'NSGA2':
        print('running: NSGA2')
        algorithm = NSGA2(
            pop_size=pop_size,
            sampling=FloatRandomSampling(),
            crossover=crossover_op,
            mutation=mutation_op,
        )
    
    if algorithm_name == 'RNSGA2' : 
        print('running: RNSGA2')
        ref_points = ref_points
        print(ref_points)
        algorithm = RNSGA2(
            pop_size=pop_size,
            ref_points= ref_points,
            sampling=FloatRandomSampling(),
            crossover=crossover_op,
            mutation=mutation_op,
        )
    if algorithm_name == 'RNSGA3' : 
        print('running: RNSGA3')
        ref_points = ref_points
        algorithm = RNSGA3(
            #pop_size=pop_size,
            ref_points= ref_points,
            pop_per_ref_point=pop_size,
            sampling=FloatRandomSampling(),
            crossover=crossover_op,
            mutation=mutation_op,
        )
   
    if algorithm_name == 'NSGA3' : 
        print('Running NSGA3')
        ref_dirs = get_reference_directions("energy", n_objeectives, pop_size)
        algorithm = NSGA3(
            pop_size=pop_size,
            sampling=FloatRandomSampling(),
            crossover=crossover_op,
            mutation=mutation_op,
            ref_dirs = ref_dirs
        )
    
    if algorithm_name == 'UNSGA3' : 
        print('Running NSGA3')
        ref_dirs = get_reference_directions("energy", n_objeectives, pop_size)
        algorithm = UNSGA3(
            pop_size=pop_size,
            sampling=FloatRandomSampling(),
            crossover=crossover_op,
            mutation=mutation_op,
            ref_dirs = ref_dirs
        )
    if algorithm_name == 'GA' : 
        algorithm = GA(
            pop_size=pop_size,
            sampling=FloatRandomSampling(),
            crossover=crossover_op,
            mutation=mutation_op,
            eliminate_duplicates=True)
    return algorithm


def train_MOGA(data,outcome,costs,algorithm,n_gen = 200,
                objectives = ['recall','churn_cost_predictions']): 
    
    problem = LearnLrWeights(data, outcome, costs,objectives = objectives)
    algorithm = copy.deepcopy(algorithm)    
    res = minimize(problem,
                    algorithm,
                   ('n_gen', n_gen),
                   seed=1,
                   verbose=True)
    X, F = res.opt.get("X", "F")
    return X,F,res.problem

In [ ]:
class MOGA_LR_warapper : 
    def __init__(
                 self,
                 params = {},
                 objectives = ['recall','misclassification_cost'],
                 ) : 
        
        #definition variables 
        self.X_train = None
        self.y_train = None 
        self.cost_train = None 
        self.default_params = {
            'GA_algorithm' : 'NSGA2',
            "n_gen" : 500,
            'population_size': 200,
            'crossover_op' : SBX( prob=0.9, eta=15),
            'mutation_op' : PolynomialMutation(eta=20,prob = 1/(len(FEATURES)) ),
            'ref_points' : np.array([0.0,0.0]),
            'misclassification_cost_coef' : 20 
        }
        self.actual_params = copy.deepcopy(self.default_params)
        self.set_params(params)
        #self.prediction_cost_function = prediction_cost_function
        #self.preprocessing_function = preprocessing_function
        self.objectives = objectives
        #state variables 
        self.is_fit = False
        
        self.opt_problem = None
        self.learned_weights = None
        self.weights_objectives = None 
        self.train_indicies=None 
        self.validation_indicies = None
        self.X_val = None 
        self.y_val = None  
        self.best_model_idx= None
        
    def fit(self,X,y,costs,train_indicies = None, validation_indicies = None
            ) :

        self.train_indicies=None 
        self.validation_indicies = None 

        self.X_train = X
        self.y_train = y 
        self.train_costs = costs
        if not (train_indicies is None) : 
            
            self.train_indicies = train_indicies
            self.validation_indicies = validation_indicies
            self.X_train = self.X_train.iloc[self.train_indicies]
            self.y_train = self.y_train.iloc[self.train_indicies]
            self.train_costs = costs[self.train_indicies]

            self.X_val = X.iloc[self.validation_indicies]
            self.y_val = y.iloc[self.validation_indicies]
            self.costs_val = costs[self.validation_indicies]

        #X_preprocessed, _= preprocessing_function(X)  
        self.ga_algorithm = define_algorithm(algorithm_name= self.actual_params['GA_algorithm'],
                            pop_size=self.actual_params['population_size'],
                            crossover_op = self.actual_params['crossover_op'],
                            mutation_op= self.actual_params['mutation_op'],n_objeectives=len(self.objectives),
                            ref_points=None
                            ) 
        final_X_train = self.X_train.copy()    
    
        self.learned_weights, self.weights_objectives, self.opt_problem = train_MOGA(final_X_train,np.array(self.y_train),np.array(self.train_costs),self.ga_algorithm,self.actual_params['n_gen'], self.objectives)
        #print(self.weights_objectives)
        self.is_fit = True
       
        dists = self.weights_objectives[:,1]**2 + self.weights_objectives[:,0]**2
        print(dists)
        self.best_model_idx = np.argmin(np.array(dists))
        
    def predict(self,X) : 
        #X_preprocessed,_ = self.preprocessing_function(X)
        probabilities = self.predict_proba(X)
        predictions = predict(probabilities,self.opt_problem.prediction_threshold)
        #costs = self.prediction_cost_function(X,predictions)
        return predictions[:,self.best_model_idx]



    def predict_proba(self,X) : 
        if not(self.is_fit) : 
            raise NotFittedError('GA is not fitted')
        final_X = X.copy()
        return compute_predictions_probabilities(final_X,self.learned_weights)

    def set_params(self,params) : 
        for param_name,value in params.items():
             self.actual_params[param_name] = value
             self.is_fit = False 
    

In [ ]:
#main
idx = 0
n_runs = 100
all_results = []
for train_file_name in os.listdir(DATA_PATH): 
    for irun in range(n_runs):
        if not('.csv' in train_file_name): 
            continue
        if not('train' in train_file_name): 
            continue
        idx += 1 
        train_data_df = pd.read_csv(os.path.join(DATA_PATH, train_file_name)) 
        test_data_df = pd.read_csv(os.path.join(DATA_PATH, train_file_name.replace('train', 'test')))

        X_train, y_train = train_data_df[FEATURES], train_data_df[TARGET]
        X_test, y_test = test_data_df[FEATURES], test_data_df[TARGET]
        scaler = StandardScaler()
        X_train_scaled = copy.deepcopy(X_train)
        X_train_scaled[FEATURES] = scaler.fit_transform(X_train[FEATURES])
        X_test_scaled = copy.deepcopy(X_test)
        X_test_scaled[FEATURES] = scaler.transform(X_test[FEATURES])

        MOLR_model = MOGA_LR_warapper()
        MOLR_model.fit(X_train_scaled, y_train, costs=None)
        y_test_pred = MOLR_model.predict(X_test_scaled)
        print('f1:', f1_score(y_test, y_test_pred))
        print('G:',geometric_mean_score(y_test, y_test_pred))
        print('MCC:',matthews_corrcoef(y_test, y_test_pred))
        with open(f'./{train_file_name}_run_{str(irun)}.pkl', 'wb') as f: 
            pickle.dump(MOLR_model, f)
        new_row = {
            'algorithm' : 'MOLR',
            'file_id' : train_file_name,
            'model_id': 'best_performance_model',
            'f1' :f1_score(y_test, y_test_pred), 
            'G' : geometric_mean_score(y_test, y_test_pred), 
            'MCC': matthews_corrcoef(y_test, y_test_pred),
            "project_name": train_file_name.split('_')[0]
        }
        all_results.append(new_row)
        if idx % 3 == 0: 
            pd.DataFrame(all_results).to_csv(f'{DATASET_NAME}_results.csv',index=False)